In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nandeesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nandeesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_excel('input.xlsx')

In [4]:
for index, row in df.iterrows():
  url = row['URL']
  url_id = row['URL_ID']

  # make a request to url
  header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
  try:
    response = requests.get(url,headers=header)
  except:
    print("can't get response of {}".format(url_id))

  #create a beautifulsoup object
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  #find title
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  #find text
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  #write title and text to the file
  file_name = f'C:\\Users\\Nandeesh\\BlackCoffer\\files\\{url_id}.txt'
  with open(file_name, 'w', encoding='utf-8') as file:
    file.write(title + '\n' + article)


can't get title of 44
can't get title of 57
can't get title of 144


In [5]:
from nltk.tokenize import sent_tokenize

In [6]:
text_dir = "C:\\Users\\Nandeesh\\BlackCoffer\\files"
stopwords_dir = "C:\\Users\\Nandeesh\\BlackCoffer\\stopwords"
sentment_dir = "C:\\Users\\Nandeesh\\BlackCoffer\\sentment"

stop_words = set()
for files in os.listdir(stopwords_dir):
  with open(os.path.join(stopwords_dir,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
  with open(os.path.join(text_dir,text_file),'r',encoding='utf-8') as f:
    text = f.read()
#tokenize the given text file
    words = word_tokenize(text)
# remove the stop words from the tokens
    filtered_text = [word for word in words if word.lower() not in stop_words]
# add each filtered tokens of each file into a list
    docs.append(filtered_text)



# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
  if files =='positive-words.txt':
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      pos.update(f.read().splitlines())
  else:
    with open(os.path.join(sentment_dir,files),'r',encoding='ISO-8859-1') as f:
      neg.update(f.read().splitlines())

# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words 
positive_words = []
Negative_words =[]
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

#iterate through the list of docs
for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos])
  Negative_words.append([word for word in docs[i] if word.lower() in neg])
  positive_score.append(len(positive_words[i]))
  negative_score.append(len(Negative_words[i]))
  polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
  subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))


In [7]:
avg_sentence_length = []
Percentage_of_Complex_words = []
Fog_Index = []
complex_word_count = []
avg_syllable_word_count = []

stopwords = set(nltk.corpus.stopwords.words('english'))

def measure(file):
    with open(os.path.join(text_dir, file), 'r', encoding='utf-8') as f:
        text = f.read()

        # remove punctuations 
        text = re.sub(r'[^\w\s.]','',text)

        # split the given text file into sentences
        sentences = text.split('.')

        # total number of sentences in a file
        num_sentences = len(sentences)

        # total words in the file
        words = [word  for word in text.split() if word.lower() not in stopwords ]
        num_words = len(words)

        # complex words having syllable count is greater than 2
        # Complex words are words in the text that contain more than two syllables.
        complex_words = []
        for word in words:
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word > 2:
                complex_words.append(word)

        # Syllable Count Per Word
        # We count the number of Syllables in each word of the text by counting the vowels present in each word.
        # We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
        syllable_count = 0
        syllable_words =[]
        for word in words:
            if word.endswith('es'):
                word = word[:-2]
            elif word.endswith('ed'):
                word = word[:-2]
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word >= 1:
                syllable_words.append(word)
                syllable_count += syllable_count_word

        avg_sentence_len = num_words / num_sentences
        avg_syllable_word_count = syllable_count / len(syllable_words)
        Percent_Complex_words  =  len(complex_words) / num_words
        Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

        return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
    x, y, z, a, b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_Complex_words.append(y)
    Fog_Index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)


In [8]:

# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by 
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
  with open(os.path.join(text_dir,file), 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stopwords]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
  x, y = cleaned_words(file)
  word_count.append(x)
  average_word_length.append(y)


# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
  with open(os.path.join(text_dir,file), 'r', encoding='utf-8') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
  return count

pp_count = []
for file in os.listdir(text_dir):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [9]:
pp_count

[3,
 3,
 12,
 5,
 3,
 6,
 1,
 3,
 4,
 38,
 1,
 1,
 2,
 8,
 3,
 3,
 43,
 8,
 8,
 2,
 7,
 35,
 13,
 8,
 8,
 3,
 4,
 7,
 3,
 2,
 3,
 1,
 7,
 1,
 5,
 8,
 5,
 4,
 7,
 1,
 1,
 4,
 1,
 2,
 3,
 10,
 2,
 3,
 1,
 9,
 2,
 7,
 3,
 18,
 13,
 19,
 8,
 3,
 10,
 2,
 9,
 7,
 33,
 12,
 1,
 16,
 1,
 1,
 3,
 3,
 4,
 10,
 5,
 1,
 5,
 2,
 1,
 3,
 14,
 2,
 1,
 5,
 6,
 17,
 19,
 17,
 1,
 5,
 9,
 8,
 12,
 6,
 1,
 7,
 14,
 6,
 1,
 1,
 6,
 1,
 11,
 1,
 3,
 2,
 6,
 6,
 4,
 2,
 4,
 28,
 13]

In [11]:

output_df = pd.read_excel('Output Data Structure.xlsx')

# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop([44-37,57-37,144-37], axis = 0, inplace=True)

# These are the required parameters 
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
  output_df.iloc[:,i+2] = var

#now save the dataframe to the disk
output_df.to_csv('Output_Data.csv')

In [12]:
positive_words

[['work',
  'like',
  'dominated',
  'trump',
  'Trump',
  'Won',
  'best',
  'best',
  'leading',
  'poised',
  'support',
  'low-cost',
  'support',
  'ambitious',
  'comprehensive',
  'coherent',
  'clearly',
  'sophisticated',
  'soft',
  'important',
  'intelligence',
  'innovation',
  'extraordinary',
  'Right'],
 ['like',
  'good',
  'foresight',
  'recommend',
  'best',
  'confidence',
  'intelligence',
  'innovation',
  'extraordinary',
  'Right'],
 ['like',
  'boom',
  'integral',
  'comprehensive',
  'excellent',
  'accolades',
  'smart',
  'advanced',
  'clear',
  'good',
  'helping',
  'pleasure',
  'tender',
  'healthy',
  'gained',
  'ethical',
  'satisfy',
  'good',
  'ready',
  'brilliant',
  'acumen',
  'worked',
  'perfect',
  'clear',
  'passion',
  'upgraded',
  'rapid',
  'like',
  'leading',
  'leading',
  'intelligence',
  'innovation',
  'extraordinary',
  'Right'],
 ['like',
  'like',
  'like',
  'helpful',
  'like',
  'improved',
  'like',
  'good',
  'Right'

In [13]:
Negative_words

[['stagnate',
  'reject',
  'rejection',
  'worsen',
  'rejection',
  'Retreat',
  'Virus',
  'Break',
  'Virus',
  'infection',
  'devastating',
  'collapse',
  'turmoil',
  'risks',
  'weaknesses',
  'outbreak',
  'Risk',
  'debt',
  'lost',
  'Unemployed',
  'hard',
  'loss',
  'recession',
  'vulnerable',
  'picky',
  'weaken',
  'outbreak',
  'fuzzy',
  'blurred',
  'controversial',
  'terribly',
  'cash-strapped',
  'stress',
  'disaster',
  'risks',
  'overwhelming',
  'outbreak'],
 ['decline', 'risk', 'issues'],
 ['Disorder',
  'illusion',
  'paradoxically',
  'peculiar',
  'absence',
  'reluctant',
  'grim',
  'disturbed',
  'trapped',
  'vicious',
  'bad',
  'worse',
  'struggling',
  'harm',
  'froze',
  'lost',
  'losses',
  'aggression',
  'tarnished',
  'depression',
  'risks',
  'suffering',
  'Disorder',
  'drained',
  'skinny',
  'toll',
  'disturbance',
  'sucked',
  'insensitive',
  'negative',
  'illegal',
  'risk',
  'Unfortunately',
  'miss',
  'numb',
  'stern',
